<a href="https://colab.research.google.com/github/carvalheirafc/deep_learning_stuff/blob/master/neural_networks/le_net/le_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Section


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

from keras.callbacks import ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.utils import np_utils
from keras import backend

from imutils import paths
import cv2
import numpy as np
import os
import tensorflow as tf
import random

Using TensorFlow backend.


## LeNet Class Definition![alt text](https://engmrk.com/wp-content/uploads/2018/09/LeNet_Original_Image.jpg)





In [2]:
class LeNet:
  @staticmethod
  def build(n_channels, 
            rows, 
            cols, 
            n_classes, 
            activation='relu', 
            weights_path=None):
    model = Sequential()
    
    input_Shape = (rows, cols, n_channels)
    
   
    #Convolution and Pooling Layers
    model.add(Conv2D(20, kernel_size=5, activation=activation, input_shape=input_Shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(50, kernel_size=5, activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #Flatten and Fully Conected layer
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation(activation))
    
    # OutputLayer SOFTMAX activation
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    
    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weights_path is not None:
      model.load_weights(weights_path)
    
    return model

## Reading or Import Image Files.
São Criados dois arrays, data para guardar todos os dados de entrada e labels para todas as labels.

- Listar todos os arquivos de imagem de todos os sub-diretórios.
- Percorrer a lista e fazer a leitura do arquivo de imagem.
  - convertida para modo em grayscale
  - Convertida para o formato de NumpyArray
  - Adicionada ao fim do vetor com todos os dados.
  
- Label é extraida usando o caminho da imagem.
 - Ex: root_path/Letras/0/img1.png 
 - Vai gerar a label: 0, que corresponde a letra A.

 



In [3]:
root_path = 'Letras/'

data = []
labels = []
all_images_paths = list(paths.list_images(root_path))
current = -1


try:
    for image_path in all_images_paths:
    
    
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = img_to_array(image)
        data.append(image)
    
        label = image_path.split(os.path.sep)[-2][7:]
        label = int(label)
        labels.append(label)
        
        if current != label:
            print('Loading Files into arrays...')
            print('Loading Class [{}]'.format(label))
            current = label
        
        current = label
    print('All Files are Loaded Successfully...')

except IOError:
    print('Something went Wrong loading the files')    

Loading Files into arrays...
Loading Class [0]
Loading Files into arrays...
Loading Class [1]
Loading Files into arrays...
Loading Class [10]
Loading Files into arrays...
Loading Class [11]
Loading Files into arrays...
Loading Class [12]
Loading Files into arrays...
Loading Class [13]
Loading Files into arrays...
Loading Class [14]
Loading Files into arrays...
Loading Class [15]
Loading Files into arrays...
Loading Class [16]
Loading Files into arrays...
Loading Class [17]
Loading Files into arrays...
Loading Class [18]
Loading Files into arrays...
Loading Class [19]
Loading Files into arrays...
Loading Class [2]
Loading Files into arrays...
Loading Class [20]
Loading Files into arrays...
Loading Class [21]
Loading Files into arrays...
Loading Class [22]
Loading Files into arrays...
Loading Class [23]
Loading Files into arrays...
Loading Class [24]
Loading Files into arrays...
Loading Class [25]
Loading Files into arrays...
Loading Class [3]
Loading Files into arrays...
Loading Class [

In [4]:
save_files = False

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

if save_files:
    try:
        np.save('char_input_data', data)
        np.save('labels', labels)
    except IOError:
        print('Error while saving the files')

## Model Configuration and Train Run

Alguns parâmetros usados no trein:

- Epochs: 10

- Learning Rate: 1e-3

- Batch Size: 32

In [5]:
config = tf.ConfigProto()
sess = tf.Session(config=config) 
backend.set_session(sess)

### Divisão dos conjuntos de treino e teste

#### Como já visto em sala de aula, os conjuntos devem ser de tamanhos semelhantes e totalmente disjuntos.

Logo Foi usada a função **train_test_split** para dividir corretamente a base de dados e labels nos respectivos conjuntos.
- Foi definido que cada conjunto teria cerca de 1/3 do tamanho total da base de dados. 
- Seed pseudo-randomica passada para garantir uma particionalização diferente a cada execução do código.

- Os conjuntos de de treino e teste das labels foram transformadas em arrays categóricos. Ou seja:
- Classe **0** = **array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)**

- Classe **1** = **array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)**

  ...
- Classe **25** =  **array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)**




In [6]:
seed = random.randint(1, 999)
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=seed)
cat_y_train = np_utils.to_categorical(y_train, 26)
cat_y_test = np_utils.to_categorical(y_test, 26)

In [7]:
n_epochs = 10
ini_learning_rate = 1e-3
batch_size = 32

### Test Section

- Modelo é construído usando as seguintes configurações:
  - n_channels = 1, pois a imagem só possui um canal(greyscale).
  - rows(height) = 28, largura da imagem.
  - cols(width) = 28, largura da imagem.
  - n_classes = 26, cada letra do alfabeto representada discretamente entre 0 ~ 25
  - activation = relu, Rectified Linear Activation Unit
```
if input > 0:
  return input
  else:
	return 0
```
  - weights_path=none, Inicialmente não temos nenhum conjunto de pesos para passar a rede neural.
 
- Otimizador **Adam**:
    - Diferentemente dos otimizadores convencionais que possuem um learning_rate fixo para todos os pesos e atualizações, o Adam permite uma taxa de aprendizado inicial para a rede e permite também a adaptação dessa ao longo do processo de aprendizagem.
    - lerning_rate = 1e-3
    - decay(decaimento da taxa de learning_rate) = 1e-3/10
    
 - Modelo é compilado com:
  - loss='categorical_crossentropy', usado por conta do problema de reconhecimento ser muit-classes.
  - metrics='categorical_accuracy'
  
 - A cada epoch é feita uma avaliação tendo como base a métrica **categorical_accuracy** e caso haja alguma melhora no score, um novo arquivo é salvo contendo os pesos da rede neural.
    
  


In [8]:
with tf.Session() as sess:
    train_model = LeNet.build(n_channels=1, 
                              rows=28,
                              cols=28, 
                              n_classes=26, 
                              activation='relu', 
                              weights_path=None)

    opt = Adam(lr=ini_learning_rate, decay=ini_learning_rate / n_epochs)
    train_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[categorical_accuracy])
    try:
        file_path = 'weights_Letras.h5'
        check_point = ModelCheckpoint(file_path, 
                                      monitor='categorical_accuracy', 
                                      verbose=0, 
                                      save_best_only=True,
                                      mode='max',
                                      save_weights_only=True)
        callbacks_list = [check_point]
        
        train_history = train_model.fit(x=x_train, 
                                        y=cat_y_train, 
                                        batch_size=batch_size, 
                                        epochs=n_epochs,
                                        callbacks=callbacks_list,
                                        verbose=1)
    
   
    except IOError:
        print('Error while saving the Model weights')

sess.close()

Epoch 1/10
14649/14649 [==============================] - 4s 255us/step - loss: 1.0358 - categorical_accuracy: 0.7063
Epoch 2/10
14649/14649 [==============================] - 2s 132us/step - loss: 0.2428 - categorical_accuracy: 0.9276
Epoch 3/10
14649/14649 [==============================] - 2s 136us/step - loss: 0.1713 - categorical_accuracy: 0.9530
Epoch 4/10
14649/14649 [==============================] - 2s 131us/step - loss: 0.1519 - categorical_accuracy: 0.9586
Epoch 5/10
14649/14649 [==============================] - 2s 130us/step - loss: 0.1284 - categorical_accuracy: 0.9642
Epoch 6/10
14649/14649 [==============================] - 2s 140us/step - loss: 0.1137 - categorical_accuracy: 0.9676
Epoch 7/10
14649/14649 [==============================] - 2s 137us/step - loss: 0.1023 - categorical_accuracy: 0.9719
Epoch 8/10
14649/14649 [==============================] - 2s 134us/step - loss: 0.0918 - categorical_accuracy: 0.9746
Epoch 9/10
14649/14649 [==============================] 

### Test Run

- Um novo modelo é criado com agora os pesos da rede neural gerada na fase treino.
- Esse novo modelo em teoria possui todo o conhecimento gerado pelo modelo de treino.
- Um vetor contendo as predições é gerado para que possa ser usado no cálculo das métricas.

In [9]:
save = True
with tf.Session() as sess:
    test_model = LeNet.build(n_channels=1, 
                             rows=28,
                             cols=28, 
                             n_classes=26, 
                             activation='relu', 
                             weights_path=file_path)

    pred = test_model.predict(x_test, verbose=1)
sess.close()

if save:
    np.save('predictions', pred.argmax(1))
    np.save('y_test', cat_y_test.argmax(1))

6279/6279 [==============================] - 0s 42us/step


In [10]:
precision, recall, f_betta, support = precision_recall_fscore_support(cat_y_test.argmax(1), 
                                                                      pred.argmax(1),
                                                                      average='weighted')

accuracy = accuracy_score(cat_y_test.argmax(1), pred.argmax(1))

print('Accuracy: {}'.format(accuracy))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F Betta: {}'.format(f_betta))

Accuracy: 0.9617773530817009
Precision: 0.9629782908693105
Recall: 0.9617773530817009
F Betta: 0.96180078775145
